In [1]:
import sys
import os
import pandas as pd
sys.path.append('/afs/cern.ch/cms/PPD/PdmV/tools/McM/')
from rest import McM

In [8]:
"""
base = '../EXO-MCsampleRequests/genFragments/Hadronizer/13p6TeV/LLstau'
for pythia_file in os.listdir(base):
    with open(os.path.join(base, pythia_file), 'r', encoding='utf-8') as f:
        content = f.read()
        
    content = content.replace(
        "'/eos/cms/store/user/fiorendi/mc_sus/run3/sms_tstaustau/SMS-TStauStau-MM_mStau-%s_slc7_amd64_gcc10_CMSSW_12_4_8_tarball.tar.xz'",
        "'/cvmfs/cms.cern.ch/phys_generator/gridpacks/RunIII/13p6TeV/slc7_amd64_gcc10/madgraph/sms_tstaustau/SMS-TStauStau-MM_mStau-%s_slc7_amd64_gcc10_CMSSW_12_4_8_tarball.tar.xz'"
    )
    with open(os.path.join(base, pythia_file), 'w', encoding='utf-8') as f:
        f.write(content)
"""

### McM scripting [tutorial](https://indico.cern.ch/event/894961/contributions/3774016/attachments/2005333/3349474/McM_Scripting_2020_03_18.pdf)
### Documentation: [restapi](https://cms-pdmv.cern.ch/mcm/restapi)

In [3]:
mcm = McM(dev=False)
tags = '20240229_dazhang_Sphaleron'
results = mcm.get('requests', query=f'tags={tags}')
len(results)

[2024-03-02 05:04:32,226][INFO] Using SSO cookie file /afs/cern.ch/user/d/dfu/private/mcm-prod-cookie.txt


60

In [13]:
CLEAR=False
if CLEAR:
    for r in results:
        mcm.delete(object_type='requests', object_id=r['_id'])

In [4]:
for r in results:
    #if r['dataset_name'].split('_')[1] in ('MStau-100', 'MStau-400'):
    #    print(r)
    #    r['tags'].append('20231211_fiorendi_GMSB_staus_to_taus_high_priority')
    #r['member_of_campaign'] = r['member_of_campaign'].replace('wmLHEGEN', 'GEN')
    r['fragment'] = r['fragment'].replace('generateConcurrently = cms.untracked.bool(False)', 'generateConcurrently = cms.untracked.bool(True)')
    #r['time_event'] = [13]
    #r['size_event'] = [1252]
    #r['mcdb_id'] = 0
    #r['tags'] = ['20231211_fiorendi_GMSB_staus_to_taus']
    #print(r['dataset_name'].split('_M')[1].split('_')[0], r['fragment'].split("GRIDPACK = \'")[1].split("\' %")[0])

In [5]:
for r in results:
    output = mcm.update('requests', r)
    if output['results'] == False:
        print(r['_id'], output['message'])

In [6]:
time = set()
for r in results:
    time.add(r['time_event'][0])
time

{8}

In [6]:
r = results[0]
r['fragment']

'import FWCore.ParameterSet.Config as cms\n\nexternalLHEProducer = cms.EDProducer("ExternalLHEProducer",\n    args = cms.vstring(\'/cvmfs/cms.cern.ch/phys_generator/gridpacks/RunIII/13p6TeV/el8_amd64_gcc10/BaryoGen/Sphaleron/BaryoGEN_COM13600_thr8500_pNCS0_gridpack.tar.xz\'),\n    nEvents = cms.untracked.uint32(5000),\n    numberOfParameters = cms.uint32(1),\n    outputFile = cms.string(\'cmsgrid_final.lhe\'),\n    generateConcurrently = cms.untracked.bool(True),\n    scriptName = cms.FileInPath(\'GeneratorInterface/LHEInterface/data/run_generic_tarball_cvmfs.sh\')\n)\nimport FWCore.ParameterSet.Config as cms\n\nfrom Configuration.Generator.Pythia8CommonSettings_cfi import *\nfrom Configuration.Generator.MCTunesRun3ECM13p6TeV.PythiaCP5Settings_cfi import *\nfrom Configuration.Generator.PSweightsPythia.PythiaPSweightsSettings_cfi import *\n\ngenerator = cms.EDFilter("Pythia8ConcurrentHadronizerFilter",\n    maxEventsToPrint = cms.untracked.int32(1),\n    pythiaPylistVerbosity = cms.untr